# Modeling

## Importando bibliotecas

In [ ]:
import pickle
import pathlib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

## Preparando os dados

In [42]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
clean_data_path = DATA_DIR / 'processed' / 'ames_clean.pkl'
with open(clean_data_path, 'rb') as file:
    data = pickle.load(file)

In [43]:
data.head()

,MS.SubClass,MS.Zoning,Lot.Frontage,Lot.Area,Lot.Shape,Land.Contour,Lot.Config,Land.Slope,Neighborhood,Bldg.Type,...,Sale.Type,Sale.Condition,SalePrice,Condition,HasShed,HasAlley,Exterior,Garage.Age,Remod.Age,House.Age
0,20,RL,141.0,31770.0,IR1,Lvl,Corner,Gtl,NAmes,1Fam,...,GroupedWD,Normal,5.332438,Norm,False,False,BrkFace,50.0,50.0,50.0
1,20,RH,80.0,11622.0,Reg,Lvl,Inside,Gtl,NAmes,1Fam,...,GroupedWD,Normal,5.021189,Roads,False,False,VinylSd,49.0,49.0,49.0
2,20,RL,81.0,14267.0,IR1,Lvl,Corner,Gtl,NAmes,1Fam,...,GroupedWD,Normal,5.235528,Norm,False,False,Wd Sdng,52.0,52.0,52.0
3,20,RL,93.0,11160.0,Reg,Lvl,Corner,Gtl,NAmes,1Fam,...,GroupedWD,Normal,5.387390,Norm,False,False,BrkFace,42.0,42.0,42.0
4,60,RL,74.0,13830.0,IR1,Lvl,Inside,Gtl,Gilbert,1Fam,...,GroupedWD,Normal,5.278525,Norm,False,False,VinylSd,13.0,12.0,13.0


In [44]:
X = data.drop('SalePrice', axis=1) # features
Y = data['SalePrice'] # target

In [45]:
X_modelo = pd.get_dummies(X, drop_first=True).copy()

## Train Test Split

In [46]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_modelo,
    Y,
    test_size=0.2,
    random_state=42,
)

## Modelos - Treinamento (Escolha de modelos e hiperparâmetros)

### 1 - Regressão Linear

In [47]:
# Inicializar o modelo de Regressão Linear
modelo_linear = LinearRegression()

# Ajustar o modelo aos dados de treinamento
modelo_linear.fit(X_train, Y_train)

# Função de scoring personalizada usando o Erro Médio Quadrático (MSE)
scoring = make_scorer(mean_squared_error, greater_is_better=False)

# Realizar validação cruzada com 5 divisões usando o MSE negativo como métrica
scores_cv = cross_val_score(modelo_linear, X_modelo, Y, cv=5, scoring=scoring)

### 2 - Random Forest Regressor

In [48]:
# Inicializar o modelo Random Forest Regressor
modelo_rf = RandomForestRegressor(random_state=42)

# Definir o grid de parâmetros para ajuste do modelo
grid_parametros_rf = {
    'n_estimators': [100, 200, 300],    # Número de árvores na floresta
    'max_depth': [10, 20, 30, None],   # Profundidade máxima das árvores
    'min_samples_split': [2, 5, 10],   # Mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 4],     # Mínimo de amostras em uma folha
    'bootstrap': [True, False],        # Usar ou não amostras bootstrap
}

# Inicializar o GridSearchCV para busca de hiperparâmetros
busca_grid_rf = GridSearchCV(estimator=modelo_rf, param_grid=grid_parametros_rf, cv=3, 
                             scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Ajustar o GridSearchCV aos dados de treinamento
busca_grid_rf.fit(X_train, Y_train)

# Recuperar o melhor modelo e seus hiperparâmetros
melhor_modelo_rf = busca_grid_rf.best_estimator_
print("Melhores Hiperparâmetros:", busca_grid_rf.best_params_)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
Melhores Hiperparâmetros: {'bootstrap': True, 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


### 3 - Gradient Boosting Regressor

In [49]:
# Inicializar o modelo Gradient Boosting Regressor
modelo_boosting = GradientBoostingRegressor()

# Definir o grid de parâmetros para ajuste do modelo
grid_parametros = {
    'n_estimators': [100, 200, 300],   # Número de árvores no modelo
    'learning_rate': [0.01, 0.1, 0.2], # Taxa de aprendizado
    'max_depth': [3, 5, 7],            # Profundidade máxima das árvores
    'subsample': [0.8, 1.0],           # Fração dos dados usada em cada árvore
}

# Inicializar o GridSearchCV para busca de hiperparâmetros
busca_grid = GridSearchCV(estimator=modelo_boosting, param_grid=grid_parametros, cv=3, 
                          scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Ajustar o GridSearchCV aos dados de treinamento
busca_grid.fit(X_train, Y_train)

# Recuperar o melhor modelo e seus hiperparâmetros
melhor_modelo_boosting = busca_grid.best_estimator_
print("Melhores Hiperparâmetros:", busca_grid.best_params_)

Fitting 3 folds for each of 54 candidates, totalling 162 fits
Melhores Hiperparâmetros: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.8}


## Predição - Testando os modelos

### 1 - Regressão Linear

In [50]:
# Calcular o MSE médio dos resultados da validação cruzada (invertendo o sinal porque usamos MSE negativo)
mean_mse = -scores_cv.mean()

# Calcular o RMSE a partir do MSE (tira-se a raiz quadrada do MSE)
rsme_ = np.sqrt(mean_mse)
print(f"MSE: {mean_mse:.6f} | RMSE: {rsme_:.6f}")

MSE: 0.003134 | RMSE: 0.055982


In [51]:
# Calcular o erro percentual médio com base no RMSE
erro = 100 * (10**rsme_ - 1)
print(f'Erro médio: {erro:.2f}%')

Erro médio: 13.76%


### 2 - Random Forest Regressor

In [52]:
# Gerar predições no conjunto de teste usando o melhor modelo de Random Forest
y_pred_rf = melhor_modelo_rf.predict(X_test)

# Calcular o MSE comparando as predições com os valores reais
mse_rf = mean_squared_error(Y_test, y_pred_rf)

# Calcular o RMSE com base no MSE
rsme_ = np.sqrt(mse_rf)
print(f"MSE: {mse_rf:.6f} | RMSE: {rsme_:.6f}")

MSE: 0.003387 | RMSE: 0.058202


In [53]:
# Calcular o erro percentual médio com base no RMSE
erro = 100 * (10**rsme_ - 1)
print(f'Erro médio: {erro:.2f}%')

Erro médio: 14.34%


### 3 - Gradient Boosting Regressor

In [54]:
# Gerar predições no conjunto de teste usando o melhor modelo de Gradient Boosting
y_pred = melhor_modelo_boosting.predict(X_test)

# Calcular o MSE comparando as predições com os valores reais
mse = mean_squared_error(Y_test, y_pred)

# Calcular o RMSE com base no MSE
rsme_ = np.sqrt(mse)
print(f"MSE: {mse:.6f} | RMSE: {rsme_:.6f}")

MSE: 0.002734 | RMSE: 0.052291


In [55]:
# Calcular o erro percentual médio com base no RMSE
erro = 100 * (10**rsme_ - 1)
print(f'Erro médio: {erro:.2f}%')

Erro médio: 12.80%


## Escolha do melhor modelo (para este caso)

Após a análise detalhada dos resultados obtidos com os três modelos de regressão, o **Gradient Boosting Regressor** foi escolhido como a opção mais adequada para este projeto, devido ao seu desempenho consistente e superior nas principais métricas de avaliação. Abaixo, apresentamos uma tabela com os valores comparativos de MSE (Mean Squared Error), RMSE (Root Mean Squared Error) e erro percentual para cada modelo:

Index | Modelo | MSE | RMSE | Error |
--- | --- | --- | --- | --- |
 1 | Regressão Linear | 0.003134 | 0.055982 | 13.76% |
 2 | Random Forest Regressor | 0.003387 | 0.058202 | 14.34% |
 3 | Gradient Boosting Regressor | 0.002734 | 0.052291 | 12.80% |

### Justificativa para a Escolha do Gradient Boosting Regressor:

1. **Menor MSE (Mean Squared Error)**  
   O Gradient Boosting Regressor apresentou o menor MSE entre os modelos analisados, com um valor de **0.002734**. Isso indica que, em média, os erros ao quadrado foram os menores, sugerindo maior precisão do modelo nas predições.

2. **Menor RMSE (Root Mean Squared Error)**  
   O RMSE, que mede o desvio médio das predições em relação aos valores reais, foi de **0.052291** para o Gradient Boosting Regressor, demonstrando menor dispersão nos erros quando comparado aos demais modelos.

3. **Erro Percentual Mais Baixo**  
   O Gradient Boosting Regressor também apresentou o menor erro percentual, com um valor de **12.80%**, superando a Regressão Linear (**13.76%**) e o Random Forest Regressor (**14.34%**). Isso reflete sua maior precisão em relação às outras abordagens.

4. **Equilíbrio entre Complexidade e Desempenho**  
   Embora modelos como a Regressão Linear sejam mais simples e fáceis de implementar, e o Random Forest Regressor ofereça boa precisão, o Gradient Boosting Regressor se destacou por equilibrar a complexidade com um desempenho superior. Ele combina múltiplos modelos base de maneira otimizada, capturando nuances nos dados que os outros modelos não foram capazes de aproveitar.

---

### Conclusão:
Com base nos resultados das métricas analisadas, o **Gradient Boosting Regressor** foi selecionado como o modelo mais adequado para o projeto. Ele oferece o melhor equilíbrio entre precisão, robustez e capacidade de generalização, tornando-o a escolha ideal para garantir previsões confiáveis e um desempenho consistente.

## Importância das Features e Suas Implicações  

A análise das features mais importantes identificadas pelo **Gradient Boosting Regressor** ajuda a compreender quais fatores têm maior impacto no preço dos imóveis. Isso é crucial para orientar tanto a modelagem quanto as decisões de negócios. Para visualizar as importâncias, organizamos as features em ordem decrescente de relevância e destacamos as 10 mais significativas.  

In [59]:
# Criação de um DataFrame para armazenar as importâncias das features
importancias = pd.DataFrame({'Importância': melhor_modelo_boosting.feature_importances_}, index=X_train.columns)

# Ordenação das features pela importância
importancias_ordenadas = importancias.sort_values('Importância', ascending=False)

# Seleção das 10 features mais importantes
top_10_importantes = importancias_ordenadas.head(10)
print("Top 10 Features Mais Importantes:")
print(top_10_importantes)

Top 10 Features Mais Importantes:
               Importância
House.Age         0.245621
Gr.Liv.Area       0.219310
Garage.Cars       0.074036
Total.Bsmt.SF     0.058891
Exter.Qual_TA     0.053520
Fireplaces        0.036934
X1st.Flr.SF       0.035347
Garage.Area       0.031935
BsmtFin.SF.1      0.025542
Remod.Age         0.025011


A seguir, analisamos as duas principais features e suas implicações para a aplicação de negócios:  


#### **1. House.Age (Idade da Casa)**  
- **Importância:** 0.245621  
- **Implicações:**  
  A idade da casa é a feature mais importante no modelo. Casas mais novas tendem a apresentar preços mais elevados, uma vez que demandam menos manutenção e costumam ser mais modernas. No contexto de negócios, isso indica uma oportunidade de focar em propriedades novas ou reformadas, que são mais atraentes para potenciais compradores. Além disso, realizar reformas em imóveis mais antigos pode ser uma estratégia eficaz para aumentar o preço de venda.

#### **2. Gr.Liv.Area (Área Útil da Casa)**  
- **Importância:** 0.219310  
- **Implicações:**  
  A área útil da casa, ou o espaço habitável em metros quadrados, é intuitivamente uma das principais determinantes do preço. Casas maiores frequentemente têm valores mais altos no mercado. Essa informação sugere que maximizar o espaço útil em imóveis, como por meio de reformas ou ampliações, pode agregar valor. Também pode orientar estratégias de marketing, como destacar o tamanho da área útil ao promover propriedades.

#### Exemplo de Dados: Feature **Remod.Age**  

Abaixo, um exemplo dos valores da feature **Remod.Age** (Idade da última reforma), que também apareceu entre as 10 mais importantes:  

In [60]:
data["Remod.Age"]

0       50.0
1       49.0
2       52.0
3       42.0
4       12.0
        ... 
2925    22.0
2926    23.0
2927    14.0
2928    31.0
2929    12.0
Name: Remod.Age, Length: 2877, dtype: float64

### Consequências do Desempenho do Modelo Final para a Aplicação de Negócios  

Os insights derivados das features mais importantes podem direcionar ações estratégicas para melhorar o valor das propriedades e otimizar esforços de marketing e vendas.  

#### Principais Consequências:  

1. **Imóveis Novos ou Reformados**  
   A idade da casa e da última reforma têm grande impacto no preço. Isso indica que reformar imóveis antigos ou investir em propriedades novas pode trazer um retorno financeiro significativo.  

2. **Maximização de Espaços**  
   Features relacionadas ao espaço, como **Gr.Liv.Area** e **Total.Bsmt.SF**, mostram que propriedades maiores ou com espaços bem utilizados tendem a ter maior valor de mercado. Reformas que aumentem a área útil, como a conversão de porões em espaços habitáveis, podem agregar valor considerável.  

3. **Comodidades Valorizadas**  
   Features como **Garage.Cars** (vagas na garagem) e **Fireplaces** (lareiras) também se destacam. Incorporar ou melhorar essas comodidades pode justificar preços mais altos e tornar as propriedades mais atrativas para os compradores.  

4. **Qualidade da Construção e Aparência Externa**  
   A qualidade da construção e a aparência externa, representadas por features como **Exter.Qual_TA**, também são cruciais. Investir na melhoria do exterior das casas pode aumentar o valor percebido pelos compradores e acelerar as vendas.
     

---

### Conclusão:  

De maneira geral, as features mais importantes sugerem que o valor dos imóveis em Ames, Iowa, é fortemente influenciado por fatores como idade, espaço útil, comodidades e qualidade da construção. Esses insights podem ser usados para:  
- Planejar melhorias em propriedades antigas;  
- Estruturar estratégias de marketing que destacam os pontos mais valorizados pelos compradores;  
- Ajustar preços com base em características específicas dos imóveis.  

Essa análise permite decisões mais informadas e aumenta a eficácia na comercialização dos imóveis, maximizando o retorno financeiro para os envolvidos.

---